In [1]:
import itertools as it
import os
import pandas as pd
import numpy as np
import random
import vitaldb
from pyvital.pyvital import arr
import pickle
import matplotlib.pyplot as plt
import scipy.stats
import statsmodels.api as sm
import time, datetime
import neurokit2 as nk

SRATE = 300
LEN_INPUT = 60
OVERLAP = 10
LEN_PER_PRE = 60
LEN_PER_POST = 60


# 피크 사이 wave를 모두 같은 length로 만들기 위한 함수
def linear_connection(list, idx):
    int_idx = int(idx)
    return list[int_idx] + (list[int_idx+1] - list[int_idx]) * (idx - int_idx)

def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T
## 60초짜리 inputp에 대해 f 그려보기


print(datetime.datetime.now())

2022-11-04 12:38:11.544351


# Download Vital Data

In [ ]:
SRATE = 300
file_path = f'vital_to_np_pd_{SRATE}Hz'

print('loading vital data...')
start = time.time()

# tracks to extract / VENT_SET_TV -> VENT_INSP_TM, SET_INSP_TM
track_names = ["SNUADC/ECG_II", "SNUADC/PLETH", "Solar8000/VENT_INSP_TM", "Primus/SET_INSP_TM", "Orchestra/PPF20_CE", "Orchestra/RFTN20_CE", "Solar8000/NIBP_MBP", "Solar8000/ART_MBP", "Solar8000/HR", "Solar8000/ETCO2"]

# create saving folder
#file_path = "vital_to_np"
if not os.path.exists(file_path):
    os.mkdir(file_path)

# dataframe of patient information    
df = pd.read_csv("https://api.vitaldb.net/cases")

# target patients' caseids
caseids = list(vitaldb.caseids_tiva & set(df.loc[df['ane_type'] == 'General', 'caseid']))


cnt = 0
non_intu, non_mbp, non_hr, emptyCO2 = [], [], [], []
for caseid in caseids:
    cnt = cnt + 1
    print(f'{cnt}/{len(caseids)}({caseid})', end='...')

    # check if file is already existing
    filename = f'{file_path}/{caseid}.npz'
    if os.path.isfile(filename):
        print('already existing')
        continue


    # get vital file and save as numpy
    vf = vitaldb.VitalFile(caseid, track_names)
    vals = vf.to_numpy(track_names, interval=1/SRATE)

    # intubation time - find the first t which satisfies vent_set_tm != nan & ppf_ce != nan
    t_intu = np.where(~np.isnan(vals[:,5]))[0][0]

    if not np.mean(~np.isnan(vals[:,2])):
        if not np.mean(~np.isnan(vals[:,3])):
            print(f'no valid data for insp_tm')

        intu = vals[:,3]
        intv = 850 *(SRATE/100)  # maximum interval for "Primus/SET_INSP_TM"
    else:
        intu = vals[:,2]
        intv = 250 *(SRATE/100)  # maximum interval for "Solar8000/VENT_INSP_TM"

    idc_intu = np.where(~np.isnan(intu))[0]
    while True:
        if t_intu >= len(vals[:,0]):
            non_intu.append(caseid)
            print('no valid intubation time')
            break

        # vent_insp_tm이 nan이 아닌 경우
        if not np.isnan(intu[t_intu]):
            idx = np.where(idc_intu==t_intu)[0][0]
            if idx + 10*(SRATE/100) >= len(idc_intu):
                non_intu.append(caseid)
                print('...no valid intubation time')
                break                
            prev = t_intu

            # 모여있는 vent_insp_tm 길이 계산
            switch = True
            for i in range(1,11):
                if idc_intu[idx+i] - prev > intv:
                    switch = False
                prev = idc_intu[idx+i]

            if switch:
                break
                # 초반에 vent_insp_tm이 예외적으로 측정된 경우 제외    
                # if abs(t_intu-t_etco2) < 5.5*60*SRATE:
                #    break
                # else:
                #    t_intu += 1
            else:
                t_intu += 1
        else:
            t_intu += 1

    # No valid data for input
    if t_inut - SRATE * 120 < 0:
    print('no data from intubation - 120s')
    continue        
    

    # MBP value
    if not np.mean(~np.isnan(vals[:,6])):
        if not np.mean(~np.isnan(vals[:,7])):
            non_mbp.append(caseid)
            print(f'no valid data for MBP')
    nibp = vals[:,6]
    art = vals[:,7]
    mbp = np.array([art[i] if art[i]>30 else nibp[i] for i in range(len(nibp))])

    # HR
    if not np.mean(~np.isnan(vals[:,8])):
        non_hr.append(caseid)
        print('no valid data for HR')
    hr = vals[:,8]

    # non-event data : extract vital from previous 120s-60s from intubation
    ppg = vals[:,1]
    prev_ppg = ppg[t_intu - SRATE*120:t_intu - SRATE*60]

    ecg = vals[:,0]
    prev_ecg = ecg[t_intu - SRATE*120:t_intu - SRATE*60]

    nppf = vals[:,4]
    nppf = nppf[t_intu - SRATE*120:t_intu - SRATE*60]

    nrftn = vals[:,5]
    nrftn = nrftn[t_intu - SRATE*120:t_intu - SRATE*60]

    nmbp = mbp[t_intu - SRATE*120:t_intu - SRATE*60]
    nhr = hr[t_intu - SRATE*120:t_intu - SRATE*60]

    # after intubation, pain calculated using TSS, CISA
    post_ppg = ppg[t_intu:t_intu + SRATE*LEN_PER_POST]
    post_ecg = ecg[t_intu:t_intu + SRATE*LEN_PER_POST]

    ppf = vals[:,4]
    ppf = ppf[t_intu:t_intu + SRATE*LEN_PER_POST]

    rftn = vals[:,5]
    rftn = rftn[t_intu:t_intu + SRATE*LEN_PER_POST]

    embp = mbp[t_intu:t_intu + SRATE*LEN_PER_POST]
    ehr = hr[t_intu:t_intu + SRATE*LEN_PER_POST]
    
    np.savez(filename, nECG=prev_ecg, nPPG=prev_ppg, ECG=post_ecg, PPG=post_ppg, nPPF = nppf, nRFTN = nrftn, PPF=ppf, RFTN=rftn, nMBP=nmbp, MBP=embp, nHR=nhr, HR=ehr)
    print('  completed')

end = time.time()
f = open(f'{file_path}/README.txt', 'w')
f.write(f'no valid intubation time: {non_intu}\n')
f.write(f'no valid MBP: {non_mbp}\n')
f.write(f'no valid HR: {non_hr}\n')
#f.write(f'empty ETCO2: {emptyCO2}\n')
f.write(f'total time: {end-start:.2f} sec')
f.close()

# Preprocess

## Peak detection

In [5]:
print(datetime.datetime.now())
start = time.time()

# dataframe to save preprocessing info
column_list = ['caseid'] + ['1', '2']
df_preprocess = pd.DataFrame(columns = column_list)

# set variables
file_path = f'vital_to_np_pd_{SRATE}Hz'
caseids = os.listdir(file_path)

error_list = []
no_peaks = []
f_num = 0
initial = f_num
interval = len(caseids)


# create new folders
if not os.path.exists(f'cache/peaks/PPG_{SRATE}Hz_1min_seg'):
    os.mkdir(f'cache/peaks/PPG_{SRATE}Hz_1min_seg')
if not os.path.exists(f'cache/peaks/ECG_{SRATE}Hz_1min_seg'):
    os.mkdir(f'cache/peaks/ECG_{SRATE}Hz_1min_seg')


for caseid in caseids[initial:initial+interval]:
    caseid = caseid[:-4]
    f_num += 1
    print('\n###Input', f_num,'/ '+str(len(caseids))+': '+caseid+'###')


    # vital data 불러오기
    try:
        vals = np.load(f'{file_path}/{caseid}.npz')
    
    except Exception as e:
        continue

        
    if not np.mean(~np.isnan(vals['RFTN'])):
        print('no RFTN value')
        continue

    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'caseid'] = caseid

    ppg_cache = f"cache/peaks/PPG_{SRATE}Hz_1min_seg/" + caseid
    ecg_cache = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid    

    
    ### before intubation event (-120 ~ -60sec) preprocessing  
    for i in range(1):
        # vital data
        seg_ppg = vals['nPPG']
        seg_ecg = vals['nECG']

        ## 1. 결측치 제거 ##
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            print(' too much missing data', end='...')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_b60s'):
            _, ppg_peak = pickle.load(open(ppg_cache+'_b60s', 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_b60s', 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                signals, info = nk.ecg_peaks(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), sampling_rate = SRATE)
                ecg_peak = info["ECG_R_Peaks"]

            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                continue


            if len(ppg_peak)==0 or len(ecg_peak)==0:
                print('no peak')
                no_peaks.append(caseid)
                continue

            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_b60s', 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_b60s', 'wb'))
            print('...saved peak...', end='')


    ### after intubation event (0 ~ +60sec) preprocessing  
    for i in range(1):
        # vital data
        seg_ppg = vals['PPG']
        seg_ecg = vals['ECG']

        ## 1. 결측치 제거 ##
        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE

        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]

        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            print(' too much missing data', end='...')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_a60s'):
            _, ppg_peak = pickle.load(open(ppg_cache+'_a60s', 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_a60s', 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                signals, info = nk.ecg_peaks(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), sampling_rate = SRATE)
                ecg_peak = info["ECG_R_Peaks"]

            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                continue


            if len(ppg_peak)==0 or len(ecg_peak)==0:
                print('no peak')
                no_peaks.append(caseid)
                continue

            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_a60s', 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_a60s', 'wb'))
            print('...saved peak...', end='')
    
    
print(datetime.datetime.now())
print(f'total time : {time.time() - start:.3f} sec') 

2022-11-04 12:50:52.463564

###Input 1 / 2701: 5974###
...loaded peak......loaded peak...
###Input 2 / 2701: 801###
...loaded peak......loaded peak...
###Input 3 / 2701: 2790###
...loaded peak......loaded peak...
###Input 4 / 2701: 1413###
...loaded peak......loaded peak...
###Input 5 / 2701: 4057###
...loaded peak......loaded peak...
###Input 6 / 2701: 1698###
...loaded peak......loaded peak...
###Input 7 / 2701: 2084###
...loaded peak......loaded peak...
###Input 8 / 2701: 845###
...loaded peak......loaded peak...
###Input 9 / 2701: 83###
...loaded peak......loaded peak...
###Input 10 / 2701: 5976###
...loaded peak......loaded peak...
###Input 11 / 2701: 3923###
...loaded peak......loaded peak...
###Input 12 / 2701: 1349###
...loaded peak......loaded peak...
###Input 13 / 2701: 3907###
...loaded peak......loaded peak...
###Input 14 / 2701: 4108###
...loaded peak......loaded peak...
###Input 15 / 2701: 263###
...loaded peak......loaded peak...
###Input 16 / 2701: 4885###
...loaded pea

In [6]:
len(error_list), len(no_peaks)

(94, 0)

## Denoising

### Bandpass filter

In [ ]:
def ECG_filter(seg, srate):
    sos = scipy.signal.butter(5, [1,40], 'bandpass', output='sos', fs=srate)
    return scipy.signal.sosfilt(sos, seg)



## Lowess filter

## Signal Quality Assessment

### Average coefficient method

In [ ]:
# path for cache
if not os.path.exists('./cache'):
    os.mkdir('./cache')
if not os.path.exists('./cache/peaks'):
    os.mkdir('./cache/peaks')
if not os.path.exists(f"cache/peaks/PPG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/PPG_{SRATE}Hz_1min_seg")
if not os.path.exists(f"cache/peaks/ECG_{SRATE}Hz_1min_seg"):
    os.mkdir(f"cache/peaks/ECG_{SRATE}Hz_1min_seg")        
if not os.path.exists('./cache/preprocess'):
    os.mkdir('./cache/preprocess')


# dataframe to save preprocessing info
column_list = ['caseid'] + ['1', '2']
df_preprocess = pd.DataFrame(columns = column_list)


# set variables
caseids = os.listdir(file_path)
error_list = []
f_num = 0
initial = f_num
interval = len(caseids)

  
for caseid in caseids[initial:initial+interval]:
    caseid = caseid[:-4]
    f_num += 1
    print('\n###Input', f_num,'/ '+str(len(caseids))+': '+caseid+'###')


    # vital data 불러오기    
    vals = np.load(f'{file_path}/{caseid}.npz')

    if not np.mean(~np.isnan(vals['RFTN'])):
        print('no RFTN value')
        continue

    #dataframe에 새로운 행 만들기
    df_preprocess.loc[f_num-1,'caseid'] = caseid

    ppg_cache = f"cache/peaks/PPG_{SRATE}Hz_1min_seg/" + caseid
    ecg_cache = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid    
    ecg_cache2 = f"cache/peaks/ECG_{SRATE}Hz_1min_seg/" + caseid

    
    ### before intubation event (-120 ~ -60sec) preprocessing  
    for i in range(1):
        seg_ppg = vals['nPPG']
        seg_ecg = vals['nECG']

        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE


        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]


        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,'1'] = (False, nan_info, [-1, -1])
            print(' too much missing data', end='...')
            continue


        ## 2. Noise 처리 ##
        # peak detection
        if os.path.exists(ppg_cache+'_b60s'):
            _, ppg_peak = pickle.load(open(ppg_cache+'_b60s', 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_b60s', 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = sorted(list(set(ecg_peak)))


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,'1'] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_b60s', 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_b60s', 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,'1'] = (False, nan_info, [-2, -2])
            print(' too less peaks', end='')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)

        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)

        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False

        # 이 segment의 정보를 dataframe에 저장 - (전처리 성공여부, 전처리 nan 비율, 전처리 noise 비율, 통증 점수)
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info]
        df_preprocess.loc[f_num-1,'1'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}        
        print('preprocessing done...', end='')
    ##########################################################################


    ### after intubation event (0 ~ +60sec) preprocessing  
    for i in range(1):
        seg_ppg = vals['PPG']
        seg_ecg = vals['ECG']

        nan_ppg_list = np.isnan(seg_ppg)
        nan_ecg_list = np.isnan(seg_ecg)
        nan_ppg_perc = np.sum(nan_ppg_list) / LEN_INPUT / SRATE
        nan_ecg_perc = np.sum(nan_ecg_list) / LEN_INPUT / SRATE


        # ECG, PPG 둘다 결측치인 부분
        nan_both_perc = 0
        for j in range(len(seg_ppg)):
            if nan_ppg_list[j] and  nan_ecg_list[j]:
                nan_both_perc += 1
        nan_both_perc /= (LEN_INPUT*SRATE)

        # segment의 결측치 비율 정보
        nan_info = [nan_ppg_perc, nan_ecg_perc, nan_both_perc]


        # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
        if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05 or nan_both_perc > 0.05:
            df_preprocess.loc[f_num-1,'2'] = (False, nan_info, [-1, -1])
            print(' too much missing data', end='...')
            continue


        ## 2. Noise 처리 ##
        # peak detection (b60s : before 60s)
        if os.path.exists(ppg_cache+'_a60s'):
            _, ppg_peak = pickle.load(open(ppg_cache+'_a60s', 'rb'))
            ecg_peak = pickle.load(open(ecg_cache+'_a60s', 'rb'))
            print('...loaded peak...', end='')


        else:
            try:
                min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = arr.detect_qrs(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
                ecg_peak = sorted(list(set(ecg_peak)))


            except Exception as e:
                print('error of', e)
                error_list.append(caseid)
                df_preprocess.loc[f_num-1,'2'] = (False, nan_info, [-3, -3])
                continue


            if len(ppg_peak)==0:
                print('no peak')


            pickle.dump((min_peak, ppg_peak), open(ppg_cache+'_a60s', 'wb'))
            pickle.dump(ecg_peak, open(ecg_cache+'_a60s', 'wb'))
            print('...saved peak...', end='')


        # 10초 segment 내의 ppg, ecg peak idx
        #seg_ppg_min = ppg_min[(start_idx<=np.array(ppg_min)) & (np.array(ppg_min)<end_idx)]
        idx_ppg_peak = ppg_peak
        idx_ecg_peak = ecg_peak


        # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
        # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
        if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
            df_preprocess.loc[f_num-1,'2'] = (False, nan_info, [-2, -2])
            print(' too less peaks', end='')
            continue


        # 20초 segment 내의 ppg, ecg peak value
        #print(len(seg_ppg), idx_ppg_peak)
        val_ppg_peak = [seg_ppg[k] for k in idx_ppg_peak]
        val_ecg_peak = [seg_ecg[k] for k in idx_ecg_peak]

        # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1
        bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
        bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


        #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
        for k in range(len(bool_noise_ppg)):
            if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
                bool_noise_ppg[k] = True
        for k in range(len(bool_noise_ecg)):
            if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
                bool_noise_ecg[k] = True


        # 2.2 모양 이상한 noise
        # wave interval into same length(2s(200))
        len_wave = 2*SRATE
        norm_seg_ppg, norm_seg_ecg = [], []

        for k in range(len(bool_noise_ppg)):
            len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

        for k in range(len(bool_noise_ecg)):
            len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]

            # peak 사이 wave를 모두 같은 길이로 변환
            norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])


        # wave interval 사이 correlation 계산 - PPG
        mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
        mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ppg)):
            if np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1] < 0.9:
                bool_noise_ppg[k] = True
        noise_ppg_perc = np.sum(bool_noise_ppg) / len(bool_noise_ppg)

        # wave interval 사이 correlation 계산 - ECG                
        mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
        mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
        for k in range(len(bool_noise_ecg)):
            if np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1] < 0.9:
                bool_noise_ecg[k] = True
        noise_ecg_perc = np.sum(bool_noise_ecg) / len(bool_noise_ecg)

        # segment의 noise 비율 정보
        noise_info = [noise_ppg_perc, noise_ecg_perc]

        # segment를 input으로 써도 되는지
        if nan_ppg_perc < 0.05 and nan_ecg_perc < 0.05 and nan_both_perc < 0.05 and noise_ppg_perc < 0.1 and noise_ecg_perc < 0.1:
            bool_pass = True
        else:
            bool_pass = False

        # 이 segment의 정보를 dataframe에 저장 - (전처리 성공여부, 전처리 nan 비율, 전처리 noise 비율, 통증 점수)
        arry = np.empty(1, dtype=object)
        arry[0] = [bool_pass, nan_info, noise_info]
        df_preprocess.loc[f_num-1,'2'] = arry[0] #{'pass':bool_pass, 'nan_perc':nan_info, 'noise_perc':noise_info, 'tss':0, 'cisa':0}        
        print('preprocessing done...', end='')
    
         

print(f'\ndumping cache of df_preprocess {f_num}/{len(caseids)}', end='...')

# df_preprocess에 demographs(age, gender) 추가
df_demograph = pd.read_csv("https://api.vitaldb.net/cases")
df_preprocess['age'] = np.nan
df_preprocess['gender'] = np.nan

for idx, row in df_preprocess.iterrows():     
    row_demo = df_demograph[df_demograph['caseid']==int(row['caseid'])]

    df_preprocess.loc[idx, 'age'] = row_demo['age'].values[0]
    df_preprocess.loc[idx, 'gender'] = row_demo['sex'].values[0]

df_preprocess.reset_index(drop=True, inplace=True)    
pickle.dump(df_preprocess, open('cache/preprocess/df_preprocess_60s', 'wb'))
print('dumping success')

# 전처리 통과 비율 출력
nl_pass, l_pass = [], []
for idx, row in df_preprocess.iterrows():
    nl_pass.append(row['1'][0])
    l_pass.append(row['2'][0])
    
print(f'전처리 성공 비율 : intubation 직전 {np.mean(nl_pass)*100:.2f}%, intubation 직후 {np.mean(l_pass)*100:.2f}%')
print(datetime.datetime.now())

### Deep Learning Model

In [ ]:
from keras.models import Model, load_model, model_from_json
        from keras.callbacks import ModelCheckpoint, EarlyStopping
        model = model_from_json(open(f'{odir}/model.json', 'rt').read())
        model.load_weights(f'{odir}/weights.hdf5')
        model.compile(optimizer='adam', loss='mae')
        model.summary()
        model.predict(...)